<h3> Leetcode Hard problem 2| Tournament Winners | Complex SQL 8 </h3>

In [0]:
%sql
drop table if exists players;
create table players
(player_id int,
group_id int);

insert into players values (15,1);
insert into players values (25,1);
insert into players values (30,1);
insert into players values (45,1);
insert into players values (10,2);
insert into players values (35,2);
insert into players values (50,2);
insert into players values (20,3);
insert into players values (40,3);

drop table if exists matches;
create table matches
(
match_id int,
first_player int,
second_player int,
first_score int,
second_score int);

insert into matches values (1,15,45,3,0);
insert into matches values (2,30,25,1,2);
insert into matches values (3,30,15,2,0);
insert into matches values (4,40,20,5,2);
insert into matches values (5,35,50,1,1);

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
select * from players

player_id,group_id
15,1
25,1
30,1
45,1
10,2
35,2
50,2
20,3
40,3


In [0]:
%sql
select * from matches

match_id,first_player,second_player,first_score,second_score
1,15,45,3,0
2,30,25,1,2
3,30,15,2,0
4,40,20,5,2
5,35,50,1,1


<h3> Solution in sql(hive) </h3>

In [0]:
%sql

select group_id,player_id from (
select player_id,group_id,s_score,
row_number()over(partition by group_id order by group_id,s_score desc,player_id asc) as rn from players 
inner join (
select player,sum(score) as s_score from (
select first_player as player ,first_score as score from matches
union all
select second_player as player ,second_score as score from matches)a1
group by player)mat on mat.player=players.player_id)a2
where rn=1 


group_id,player_id
1,15
2,35
3,40


<h3> Solution in pyspark </h3>

In [0]:
from pyspark.sql.functions import col,sum,row_number
from pyspark.sql.window import Window

In [0]:
df_m=spark.sql("select * from matches")
df_p=spark.sql("select * from players")
df_p.show()
df_m.show()

+---------+--------+
|player_id|group_id|
+---------+--------+
|       15|       1|
|       25|       1|
|       30|       1|
|       45|       1|
|       10|       2|
|       35|       2|
|       50|       2|
|       20|       3|
|       40|       3|
+---------+--------+

+--------+------------+-------------+-----------+------------+
|match_id|first_player|second_player|first_score|second_score|
+--------+------------+-------------+-----------+------------+
|       1|          15|           45|          3|           0|
|       2|          30|           25|          1|           2|
|       3|          30|           15|          2|           0|
|       4|          40|           20|          5|           2|
|       5|          35|           50|          1|           1|
+--------+------------+-------------+-----------+------------+



In [0]:
df_m_union=df_m.select(df_m["first_player"].alias("player"),df_m["first_score"].alias("score")).unionAll\
    (df_m.select(df_m["second_player"].alias("player"),df_m["second_score"].alias("score")))
df_m_sum=df_m_union.groupBy("player").agg(sum("score").alias("score"))
df_f=df_m_sum.join(df_p,df_p.player_id==df_m_sum.player,"inner").select("group_id","player_id","score")
win=Window.partitionBy("group_id").orderBy(col("score").desc(),col("player_id").asc())
df_final=df_f.withColumn("rn",row_number().over(win))
df_final.filter(df_final.rn==1).select("group_id","player_id").show()

+--------+---------+
|group_id|player_id|
+--------+---------+
|       1|       15|
|       2|       35|
|       3|       40|
+--------+---------+

